In [ ]:
import numpy as np
import os.path as op
import pandas as pd

import mne
from mne import setup_volume_source_space
from mne.beamformer import apply_lcmv_cov, make_lcmv
from mne.cov import compute_covariance

In [ ]:
#dictionary of raw fif files per patients
raw_paths = {"patient_X": "fif file path"}

#dictionary of coregistartion files per patients
coregs_files = {"patient_X": "trans file path"}

In [ ]:
#dictionaries of epochs files
all_epochs_files = {"patient_X" : "file path"}

high_epochs_files = {"patient_X" : " file path"}

low_epochs_files = {"patient_X" : " file path"}

In [ ]:
#dictionary of selected voxels/vertices per patient
selected_vertices = {"patient_X" : [1000, 2300]}


In [ ]:
for subject, path in raw_paths.items():
    mne.set_log_level('ERROR')
    raw_fname = op.abspath( raw_paths[subject])
    raw = mne.io.read_raw_fif(raw_fname, verbose = False)
    info = raw.info

    base_subject = subject.split("_part")[0] 
    print(base_subject)
  
    
    data_path = op.abspath("path")
    subjects_dir = op.join(data_path, "freesurfer", "recons")
    coreg_fname = op.abspath(coregs_files[subject])
    coreg = mne.read_trans(coreg_fname, verbose = False)

    #improt 3 sets of epochs
    all_epochs_fname = op.abspath(all_epochs_files[subject])
    all_epochs = mne.read_epochs(all_epochs_fname, verbose = False)

    high_epochs_fname = op.abspath(high_epochs_files[subject])
    high_epochs = mne.read_epochs(high_epochs_fname, verbose = False)

    low_epochs_fname = op.abspath(low_epochs_files[subject])
    low_epochs = mne.read_epochs(low_epochs_fname, verbose = False)

    #filter in the ripple band
    all_epochs = all_epochs.filter(80, 150, verbose = False )
    high_epochs = high_epochs.filter(80, 150, verbose = False)
    low_epochs = low_epochs.filter(80, 150, verbose = False)

    #prepare source space
    surface = op.join(subjects_dir, base_subject , "bem" , "inner_skull.surf")
    src = mne.setup_volume_source_space(
    base_subject, subjects_dir=subjects_dir, pos=10.0, surface=surface, add_interpolator=False, verbose = False
    )  

    conductivity = (0.3,)
    model = mne.make_bem_model(
    subject=base_subject, ico=4, conductivity=conductivity, subjects_dir=subjects_dir, verbose = False
    )
    bem = mne.make_bem_solution(model, verbose = False)

    #compute forward solution 
    fwd = mne.make_forward_solution(
        raw_fname,
        trans=coreg,
        src=src,
        bem= bem,
        meg=True,
        eeg=False,
        mindist=5.0,
        n_jobs=None,
        verbose=False,
        )
    #prepare covariance matrix
    rank = mne.compute_rank(all_epochs, tol=1e-6, tol_kind="relative", verbose = False)
    active_win = (0, 0.3)
    baseline_win = (0, 0.3)
    common_win = (0, 0.3)

    #epochs of no SWRs
    baseline_cov = compute_covariance(
        low_epochs,
        tmin=baseline_win[0],
        tmax=baseline_win[1],
        method="shrunk",
        rank=rank,
        verbose=False,
    )
    #epochs of SWRs
    active_cov = compute_covariance(
        high_epochs,
        tmin=active_win[0],
        tmax=active_win[1],
        method="shrunk",
        rank=rank,
        verbose=False,
    )

    #epochs of both
    cov = compute_covariance(
        all_epochs,
        tmin=common_win[0],
        tmax=common_win[1],
        method="shrunk",
        rank=rank,
        verbose=False,
    )

    noise_cov = mne.make_ad_hoc_cov(all_epochs.info, verbose = False) 

    #compute the filres
    filters = make_lcmv(
        all_epochs.info, fwd, cov,
        reg=0.05, noise_cov = noise_cov,
        pick_ori="max-power", 
        weight_norm= None, 
        verbose = False
    )

    #get raw data
    raw.pick_types(meg= True)
    raw_data = raw.get_data()
    sfreq = raw.info['sfreq']

    #get vertices from beamformer
    filter_vertices = filters['vertices'][0]
    filter_weights = filters['weights']

    #find the target vertices
    if subject in selected_vertices:
        target_vertices = selected_vertices[subject]
        for vertex in target_vertices:
            try:
                # Find position of this vertex in filter_vertices
                idx = np.where(filter_vertices == vertex)[0][0]
                filt = filter_weights[idx]  # spatial filter for this vertex

                # Compute virtual channel time series - multply raw by the spatial filter
                virtual_ts = np.dot(filt, raw_data)  # shape: (n_times,)

                # Save created virtual time seares as a  MNE RawArray object
                output_dir = "path"
                ch_name = f"VE_{vertex}"
                info = mne.create_info([ch_name], sfreq=sfreq, ch_types=['misc'])
                raw_virtual = mne.io.RawArray(virtual_ts[np.newaxis, :], info)

                # Save to .fif
                fif_path = op.join(output_dir, f"{subject}_vertex{vertex}_virtual_ts.fif")
                raw_virtual.save(fif_path, overwrite=True)
                print(f"Saved: {fif_path}")

            except IndexError:
                print(f"{subject} | Vertex {vertex} not found in filters — skipping.")
    else:
        print(f"{subject} not in vertex_dict — skipping.")
    

    